In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\credit\\FraudApp\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\credit\\FraudApp'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    classifier_params: dict

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
import os
from box import Box

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        classifier_params = self.params

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name,
            classifier_params=classifier_params
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
# from sklearn.linear_model import ElasticNet
import joblib
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        X_train = train_x.values
        X_test = test_x.values
        y_train = train_y.values.ravel()
        y_test = test_y.values.ravel()

        classifiers = {
            "LogisiticRegression": LogisticRegression(),
            "KNearest": KNeighborsClassifier(),
            "Support Vector Classifier": SVC(),
            "DecisionTreeClassifier": DecisionTreeClassifier()
        }

        best_accuracy = 0.0
        best_model = None

        for classifier_name, classifier in classifiers.items():
            classifier_params = self.config.classifier_params.get(classifier_name)
            
            if classifier_params:
                grid_classifier = GridSearchCV(classifier, classifier_params)
                grid_classifier.fit(X_train, y_train)
                best_classifier = grid_classifier.best_estimator_

                training_score = cross_val_score(best_classifier, X_train, y_train, cv=5)
                accuracy = round(training_score.mean(), 2) * 100
                print("Classifier:", classifier_name, "has a training score of", accuracy, "% accuracy score")

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_model = best_classifier

        if best_model is not None:
            model_path = os.path.join(self.config.root_dir, self.config.model_name)
            joblib.dump(best_model, model_path)
            print("Best model saved with accuracy:", best_accuracy, "%")
        else:
            print("No best model found.")

        return best_model

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-08-12 20:34:17,949: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-12 20:34:17,967: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-12 20:34:17,982: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-12 20:34:17,987: INFO: common: created directory at: artifacts]
[2023-08-12 20:34:17,990: INFO: common: created directory at: artifacts/model_trainer]


Classifier: KNearest has a training score of 93.0 % accuracy score
Classifier: DecisionTreeClassifier has a training score of 91.0 % accuracy score
Best model saved with accuracy: 93.0 %
